# 8.7 通过时间反向传播
- **目录**
  - 8.7.1 循环神经网络的梯度分析
    - 8.7.1.1 完全计算
    - 8.7.1.2 截断时间步
    - 8.7.1.3 随机截断
    - 8.7.1.4 比较策略
  - 8.7.2 通过时间反向传播的细节


到目前为止，我们已经反复提到像“**梯度爆炸**”、“**梯度消失**”，
以及需要对循环神经网络“**分离梯度**”。
例如，在 8.5节中，
我们在序列上调用了`detach`函数。
为了能够快速构建模型并了解其工作原理，
上面所说的这些概念都没有得到充分的解释。
在本节中，我们将更深入地探讨序列模型反向传播的细节，
以及相关的数学原理。

当我们首次实现循环神经网络（ 8.5节）时，
遇到了梯度爆炸的问题。
如果你做了练习题，就会发现**梯度裁剪对于确保模型收敛至关重要。**
为了更好地理解此问题，本节将回顾序列模型梯度的计算方式，
它的工作原理没有什么新概念，毕竟我们使用的仍然是链式法则来计算梯度。

我们在 4.7节中描述了多层感知机中的
前向与反向传播及相关的计算图。
循环神经网络中的前向传播相对简单。
**通过时间反向传播（backpropagation through time，BPTT）**
 实际上是循环神经网络中反向传播技术的一个特定应用。
它要求我们**将循环神经网络的计算图一次展开一个时间步，
以获得模型变量和参数之间的依赖关系**。
然后，基于链式法则，应用反向传播来计算和存储梯度。
由于序列可能相当长，因此**依赖关系也可能相当长**。
例如，某个1000个字符的序列，
其第一个词元可能会对最后位置的词元产生重大影响。
这在计算上是不可行的（它需要的时间和内存都太多了），
并且还需要超过1000个矩阵的乘积才能得到非常难以捉摸的梯度。
这个过程充满了计算与统计的不确定性。
在下文中，我们将阐明会发生什么以及如何在实践中解决它们。

- **要点：**
  - 梯度爆炸和梯度消失：在循环神经网络中，我们经常遇到梯度爆炸和梯度消失的问题。为了确保模型的收敛，**梯度裁剪（gradient clipping）** 是一种必要的技术。
  - 循环神经网络的反向传播：通过时间反向传播（BPTT）是循环神经网络中的反向传播技术的一种特定应用。它通过**展开循环神经网络的计算图**，以便计算和存储梯度。
  - **展开时间步**：在BPTT中，我们**将循环神经网络的计算图展开到每个时间步，以明确模型变量和参数之间的依赖关系**。
  - 计算梯度的挑战：由于序列可能非常长，依赖关系也可能非常长。这会导致计算上的困难和梯度的不确定性。
  - 解决方法：为了解决长序列带来的计算和梯度问题，我们可以使用**截断反向传播（truncated backpropagation）**或者通过**批量处理多个较短序列来进行训练**。

## 8.7.1 循环神经网络的梯度分析


我们从一个描述循环神经网络工作原理的**简化模型**开始，
此模型忽略了隐状态的特性及其更新方式的细节。
这里的数学表示没有像过去那样明确地区分标量、向量和矩阵，
因为这些细节对于分析并不重要，
反而只会使本小节中的符号变得混乱。

在这个简化模型中，我们将时间步$t$的隐状态表示为$h_t$，
输入表示为$x_t$，输出表示为$o_t$。
回想一下我们在8.4.2节中的讨论，
**输入和隐状态可以拼接后与隐藏层中的一个权重变量相乘。**
因此，我们分别使用$w_h$和$w_o$来表示隐藏层和输出层的权重。
每个时间步的隐状态和输出可以写为：

$$\begin{aligned}h_t &= f(x_t, h_{t-1}, w_h),\\o_t &= g(h_t, w_o),\end{aligned} \tag{8.7.1}$$


其中$f$和$g$分别是隐藏层和输出层的变换。
因此，我们有一个链
$\{\ldots, (x_{t-1}, h_{t-1}, o_{t-1}), (x_{t}, h_{t}, o_t), \ldots\}$，
它们通过循环计算彼此依赖。
前向传播相当简单，<b>一次一个时间步的遍历三元组$(x_t, h_t, o_t)$</b>，
然后通过一个目标函数在所有$T$个时间步内
评估输出$o_t$和对应的标签$y_t$之间的差异：

$$L(x_1, \ldots, x_T, y_1, \ldots, y_T, w_h, w_o) = \frac{1}{T}\sum_{t=1}^T l(y_t, o_t) \tag{8.7.2}$$

对于反向传播，问题则有点棘手，
特别是当我们计算目标函数$L$关于参数$w_h$的梯度时。
具体来说，按照链式法则：

$$\begin{aligned}\frac{\partial L}{\partial w_h}  & = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial w_h}  \\& = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t, w_o)}{\partial h_t}  \frac{\partial h_t}{\partial w_h}.\end{aligned} \tag{8.7.3}$$

在公式8.7.3中乘积的第一项和第二项很容易计算，
而**第三项$\partial h_t/\partial w_h$是使事情变得棘手的地方**，
因为我们需要循环地计算参数$w_h$对$h_t$的影响。
根据公式8.7.1中的递归计算，
$h_t$既依赖于$h_{t-1}$又依赖于$w_h$，
其中$h_{t-1}$的计算也依赖于$w_h$。
因此，使用链式法则产生：

$$\frac{\partial h_t}{\partial w_h}= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h} \tag{8.7.4}$$


为了导出上述梯度，假设我们有三个序列$\{a_{t}\},\{b_{t}\},\{c_{t}\}$，
当$t=1,2,\ldots$时，序列满足$a_{0}=0$且$a_{t}=b_{t}+c_{t}a_{t-1}$。
对于$t\geq 1$，就很容易得出：

$$a_{t}=b_{t}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t}c_{j}\right)b_{i} \tag{8.7.5}$$

基于下列公式替换$a_t$、$b_t$和$c_t$：

$$\begin{aligned}a_t &= \frac{\partial h_t}{\partial w_h},\\
b_t &= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}, \\
c_t &= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}},\end{aligned} \tag{8.7.6}$$

公式 8.7.4中的梯度计算
满足$a_{t}=b_{t}+c_{t}a_{t-1}$。
因此，对于每个公式8.7.5，
我们可以使用下面的公式移除8.7.4中的循环计算

$$\frac{\partial h_t}{\partial w_h}=\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_h)}{\partial h_{j-1}} \right) \frac{\partial f(x_{i},h_{i-1},w_h)}{\partial w_h} \tag{8.7.7}$$


虽然我们可以使用链式法则递归地计算$\partial h_t/\partial w_h$，
但当$t$很大时这个链就会变得很长。我们需要想想办法来处理这一问题。

-----

- **说明：**
- **(1) 为何说“在公式8.7.3中乘积的第一项和第二项很容易计算，而第三项$\partial h_t/\partial w_h$是使事情变得棘手的地方”**
  - 公式8.7.3中乘积的第一项和第二项较容易计算，主要是因为它们都直接依赖于最后的输出$o_t$和隐藏状态$h_t$，并且这两个量在前向传播过程中已经被存储起来了。计算逻辑大致如下：
    - 第一项$\frac{\partial l(y_t, o_t)}{\partial o_t}$：这是损失$L$相对于输出$o_t$的梯度，可以直接使用预定义的损失函数（例如交叉熵损失）的求导性质得到。
    - 第二项$\frac{\partial g(h_t, w_o)}{\partial h_t}$：这是根据当前隐藏状态$h_t$和权重$w_o$计算$o_t = g(h_t, w_o)$的梯度，只需要应用我们选用的激活函数$g$的导数即可。
  - 但是，对于第三项$\frac{\partial h_t}{\partial w_h}$，其计算涉及所有时间步内的隐藏状态$h_t$，而隐藏状态$h_t$又依赖于上一个时间步的隐藏状态$h_{t-1}$，形成了复杂的循环依赖关系：
    $h_{t} = \phi(x_{t}w_x + h_{t-1}w_h)$
  - 为了计算参数$w_h$的梯度，需将每一步的隐藏状态$h_t$相对于参数$w_h$进行求导，并将这些**梯度加起来**。
  - 然而，由于隐藏状态$h_t$取决于上一步的隐藏状态$h_{t-1}$，我们实际上需要对所有之前的时间步进行反向传播，这就使得计算过程变得特别复杂。这也是长时间序列的RNN模型在训练过程中会出现梯度消失或者梯度爆炸问题的原因。


- **(2) 公式8.7.4到8.7.7的推导过程**
- 首先有以下替换定义：
  - $a_t = \frac{\partial h_t}{\partial w_h}$
  - $b_t = \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}$
  - $c_t = \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}}$

- 根据链式法则，可得：
   - $\frac{\partial h_t}{\partial w_h} = \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} + \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \cdot \frac{\partial h_{t-1}}{\partial w_h}$
- 使用上述替换，这个方程可以写成：
  - $a_t = b_t + c_t a_{t-1}$
- 递归展开这个表达式：
  - $a_t = b_t + c_t(b_{t-1} + c_{t-1}a_{t-2})$
  - $= b_t + c_tb_{t-1} + c_tc_{t-1}(b_{t-2} + c_{t-2}a_{t-3})$
  - $= b_t + c_tb_{t-1} + c_tc_{t-1}b_{t-2} + c_tc_{t-1}c_{t-2}b_{t-3} + ...$
- 或者这样展开：
  - $a_1 = b_1$ (因为$a_0 = 0$)
  - $a_2 = b_2 + c_2b_1$
  - $a_3 = b_3 + c_3(b_2 + c_2b_1) = b_3 + c_3b_2 + c_3c_2b_1$
  - 以此类推..
- 这个展开模式可以写成求和形式（公式8.7.5）：
  - $a_t = b_t + \sum_{i=1}^{t-1}\left(\prod_{j=i+1}^t c_j\right)b_i$
- 然后带入得到：
  - $\frac{\partial h_t}{\partial w_h} = \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} + \sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_h)}{\partial h_{j-1}} \right) \frac{\partial f(x_{i},h_{i-1},w_h)}{\partial w_h}$

- 推导的关键点是：
  - 使用链式法则分解复杂的导数。
  - 识别递归模式并展开。
  - 将展开的序列重写为紧凑的求和形式。
- 虽然移除了递归计算，但当序列长度t很大时，计算量仍然很大，这就是为什么在实践中需要其他技术来处理长序列的梯度计算。


----

### 8.7.1.1 完全计算

显然，我们可以仅仅计算公式8.7.7中的全部总和，
然而，**这样的计算非常缓慢，并且可能会发生梯度爆炸**，
因为初始条件的微小变化就可能会对结果产生巨大的影响。
也就是说，我们可以观察到类似于**蝴蝶效应**的现象，
即初始条件的很小变化就会导致结果发生不成比例的变化。
这对于我们想要估计的模型而言是非常不可取的。
毕竟，我们正在寻找的是能够很好地泛化高稳定性模型的估计器。
因此，在实践中，这种方法几乎从未使用过。

---------------
- **说明：何为完全计算？**
  - “完全计算”在这里指的是对梯度反向传播的**所有项**进行计算，不论其计算复杂性或可能导致的问题。
  - 在循环神经网络（RNN）中，需要计算出关于隐藏状态的梯度，并将它们传递给前一个时间步。
  - 但由于很多实际任务中的序列通常都会非常长（比如一段文字或者一段音频），因此当尝试进行“完全计算”时，也就意味着需要进行大量计算。

  - 以标准的RNN为例，设$h_t$表示t时间步的隐藏状态，$o_t$表示t时间步的输出，那么按照定义我们有$h_t = \phi(h_{t-1}, x_t)$和$o_t = f(h_t)$，其中$x_t$为输入，$\phi$和$f$分别为隐藏层和输出层的函数。
  - 对于给定的损失函数$L$，要最小化的目标就是整个序列的损失之和：$\sum_t L(o_t, y_t)$。

  - 对于每一个具体的$t$，想要计算梯度$\frac{\partial L}{\partial h_t}$，需要考虑两部分：
    - $t$时间步自身的损失对$h_t$的影响：即$\frac{\partial L_t}{\partial h_t} = \frac{\partial L_t}{\partial o_t} \cdot \frac{\partial o_t}{\partial h_t}$。
    -  所有大于$t$的时间步的损失对$h_t$的影响：即$\sum_{s=t+1}^{T}(\frac{\partial L_s}{\partial h_s} \cdot \frac{\partial h_s}{\partial h_t})$。

  - 第一部分较为直观且易于计算。然而，对于第二部分，由于每个$h_s$都依赖于它前面的隐藏状态，所以需要考虑从$t$到$s$间所有的时间步，这就使得计算变得十分复杂。另外，由于链式法则的连乘效应，当这个序列足够长或者某些参数值过大/过小时，就有可能产生梯度爆炸/消失的问题。

  - 总结来说，“完全计算”指的就是上述两部分的完整计算过程，由于涉及到复杂的循环依赖和长期累积效应，因此在实践中往往被简化或者使用特殊技术（如梯度截断、LSTM/GRU等机制）来处理。


--------

### 8.7.1.2 截断时间步

或者，我们可以**在$\tau$步后截断公式8.7.7中的求和计算**。
这是我们到目前为止一直在讨论的内容，
例如在8.5节中分离梯度时。
这会带来真实梯度的**近似**，
只需将求和终止为$\partial h_{t-\tau}/\partial w_h$。
在实践中，这种方式工作得很好。
它通常被称为**截断的通过时间反向传播**。
这样做导致该模型主要侧重于短期影响，而不是长期影响。
这在现实中是可取的，因为它会将估计值偏向更简单和更稳定的模型。

- **说明：何为截断时间步？**

  - 在RNN（循环神经网络）中，截断时间步是反向传播过程的一种简化方式，主要用于解决长期依赖问题和梯度消失/爆炸问题。

  - 以下是使用截断时间步进行反向传播的步骤：

    - 前向传播：首先执行标准的前向传播过程，即根据输入计算隐藏状态，并将其传递给下一个时间步；

    - 选择截断长度：选择一个固定的截断长度τ，这是反向传播时需要**回溯的最大时间步数**。可以通过实验调整此参数以优化性能；

    - 开始反向传播：在每个时间步t皆需要计算损失函数关于当前输出的导数，并将误差信号向前一步传播；

    - 应用截断：但是，与标准的全序列反向传播不同，此处只回溯τ步。
      - 即当从当前时间步t返回到（t-τ）时，会停止反向传播，而忽视在（t-τ）之前的任何步骤。

    - 更新权重：然后，可以基于计算出的梯度来更新模型的权重。
      - 注意，由于截断忽略了部分历史信息，所以这只是真实梯度的近似。

    - 迭代处理：以上步骤在训练数据上多次迭代执行，直至满足终止条件。

  - 请注意，虽然截断时间步可以显著降低计算负担并防止过度复杂的依赖关系，但如果截断长度设置得太小，则可能无法捕捉到更长期的依赖关系。因此，根据具体任务和数据特性来决定适合的截断长度是非常重要的。

### 8.7.1.3 随机截断

最后，我们可以用一个随机变量替换$\partial h_t/\partial w_h$，
该随机变量在预期中是正确的，但是会截断序列。
这个随机变量是通过使用序列$\xi_t$来实现的，
序列预定义了$0 \leq \pi_t \leq 1$，
其中$P(\xi_t = 0) = 1-\pi_t$且$P(\xi_t = \pi_t^{-1}) = \pi_t$，
因此$E[\xi_t] = 1$。
我们使用它来替换公式8.7.4中的
梯度$\partial h_t/\partial w_h$得到：

$$z_t= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\xi_t \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h} \tag{8.7.8}$$

从$\xi_t$的定义中推导出来$E[z_t] = \partial h_t/\partial w_h$。
每当$\xi_t = 0$时，递归计算终止在这个$t$时间步。
这导致了不同长度序列的加权和，其中长序列出现的很少，
所以将适当地加大权重。

--------

- **说明：**
  - 公式8.7.8描述的是在计算循环神经网络（RNN）的梯度时，使用随机截断策略。这个策略通过引入一个随机变量 $\xi_t$ 来截断梯度的计算序列，从而减少计算复杂性。
  - 在这个公式中，$z_t$ 表示在时间步 $t$ 的梯度；$f(x_{t},h_{t-1},w_h)$ 是RNN的输出函数，$x_{t}$ 是当前的输入，$h_{t-1}$ 是前一步的隐藏状态，$w_h$ 是权重；$\xi_t$ 是一个随机变量，它的取值决定了是否在当前时间步截断梯度的计算。
  - $\xi_t$ 的取值有两种可能，一种是0，另一种是 $\pi_t^{-1}$ 。
    - 当 $\xi_t = 0$ 时，梯度的计算在当前时间步被截断，不再向前传播；
    - 当 $\xi_t = \pi_t^{-1}$ 时，梯度的计算继续向前传播。$\xi_t$ 的取值由概率 $\pi_t$ 决定，其中 $P(\xi_t = 0) = 1-\pi_t$ ，$P(\xi_t = \pi_t^{-1}) = \pi_t$ 。


-----------

### 8.7.1.4 比较策略


<center>    <img src="../img/truncated-bptt.svg" alt="比较RNN中计算梯度的策略，3行自上而下分别为：随机截断、常规截断、完整计算">
</center>
<center>图8.7.1 比较RNN中计算梯度的策略，3行自上而下分别为：随机截断、常规截断、完整计算</center><br>

图8.7.1说明了
当基于循环神经网络使用通过时间反向传播
分析《时间机器》书中前几个字符的三种策略：

* 第一行采用随机截断，方法是将文本划分为不同长度的片断。
* 第二行采用常规截断，方法是将文本分解为相同长度的子序列。
  这也是我们在循环神经网络实验中一直在做的。
* 第三行采用通过时间的完全反向传播，结果是产生了在计算上不可行的表达式。

遗憾的是，虽然随机截断在理论上具有吸引力，
但很可能是由于多种因素**在实践中并不比常规截断更好**。
首先，在对过去若干个时间步经过反向传播后，
观测结果足以捕获实际的依赖关系。
其次，增加的方差抵消了时间步数越多梯度越精确的事实。
第三，我们真正想要的是只有短范围交互的模型。
因此，模型需要的正是截断的通过时间反向传播方法所具备的**轻度正则化效果**。

- **要点：**
  - 图8.7.1描绘了三种基于循环神经网络使用通过时间反向传播分析《时间机器》书中前几个字符的策略：随机截断、常规截断和完整计算。
  - 随机截断的方法是将文本划分为不同长度的片断。
  - 常规截断的方法是将文本分解为相同长度的子序列，这是在循环神经网络实验中常用的方法。
  - 通过时间的完全反向传播会产生在计算上不可行的表达式。
  - 尽管随机截断在理论上具有吸引力，但由于多种因素，在实践中并不比常规截断更好。
  - 一些原因包括： 
    - 对过去若干个时间步经过反向传播后，观测结果足以捕获实际的依赖关系；
    - 增加的方差抵消了时间步数越多梯度越精确的事实；
    - 我们实际上希望的是只有短范围交互的模型。
  - 截断的通过时间反向传播方法具有轻度正则化效果，这正是模型需要的。

## 8.7.2 通过时间反向传播的细节

在讨论一般性原则之后，我们看一下通过时间反向传播问题的细节。
与 8.7.1节中的分析不同，
下面我们将展示如何计算目标函数**相对于所有分解模型参数的梯度**。
为了保持简单，我们考虑一个没有偏置参数的循环神经网络，
其在隐藏层中的激活函数使用恒等映射（$\phi(x)=x$）。
对于时间步$t$，设单个样本的输入及其对应的标签分别为
$\mathbf{x}_t \in \mathbb{R}^d$和$y_t$。
计算隐状态$\mathbf{h}_t \in \mathbb{R}^h$和
输出$\mathbf{o}_t \in \mathbb{R}^q$的方式为：

$$\begin{aligned}\mathbf{h}_t &= \mathbf{W}_{hx} \mathbf{x}_t + \mathbf{W}_{hh} \mathbf{h}_{t-1},\\
\mathbf{o}_t &= \mathbf{W}_{qh} \mathbf{h}_{t},\end{aligned} \tag{8.7.9}$$

其中权重参数为$\mathbf{W}_{hx} \in \mathbb{R}^{h \times d}$、
$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$和
$\mathbf{W}_{qh} \in \mathbb{R}^{q \times h}$。
用$l(\mathbf{o}_t, y_t)$表示时间步$t$处
（即从序列开始起的超过$T$个时间步）的损失函数，
则我们的目标函数的总体损失是：

$$L = \frac{1}{T} \sum_{t=1}^T l(\mathbf{o}_t, y_t)\tag{8.7.10}$$

为了在循环神经网络的计算过程中可视化模型变量和参数之间的依赖关系，
我们可以为模型绘制一个计算图，
如图8.7.2所示。
例如，时间步3的隐状态$\mathbf{h}_3$的计算
取决于模型参数$\mathbf{W}_{hx}$和$\mathbf{W}_{hh}$，
以及最终时间步的隐状态$\mathbf{h}_2$
以及当前时间步的输入$\mathbf{x}_3$。


<center><img src='..\img\rnn-bptt.svg'></center>
<center>图8.7.2 上图表示具有三个时间步的循环神经网络模型依赖关系的计算图。<br>未着色的方框表示变量，着色的方框表示参数，圆表示运算符</center><br>

正如刚才所说，图8.7.2 中的模型参数是
$\mathbf{W}_{hx}$、$\mathbf{W}_{hh}$和$\mathbf{W}_{qh}$。
通常，训练该模型需要对这些参数进行梯度计算：
$\partial L/\partial \mathbf{W}_{hx}$、
$\partial L/\partial \mathbf{W}_{hh}$和
$\partial L/\partial \mathbf{W}_{qh}$。
根据图8.7.2中的依赖关系，
我们可以**沿箭头的相反方向遍历计算图**，依次计算和存储梯度。
为了灵活地表示链式法则中不同形状的矩阵、向量和标量的乘法，
我们继续使用如4.7节中所述的$\text{prod}$运算符。

---------

- **说明：**
  - 图8.7.2是一个计算图，它表示了一个具有三个时间步的循环神经网络模型的依赖关系。
    - 图中未着色的方框表示变量，着色的方框表示参数，圆形表示运算符。
  - 此模型中有三个参数：$\mathbf{W}_{hx}$, $\mathbf{W}_{hh}$, $\mathbf{W}_{qh}$。
    - 这些参数在所有时间步中都是共享的，这是循环神经网络的一大特点。
  - 以时间步3为例，$\mathbf{h}_3$的计算取决于模型参数$\mathbf{W}_{hx}$和$\mathbf{W}_{hh}$，以及最终时间步的隐状态$\mathbf{h}_2$以及当前时间步的输入$\mathbf{x}_3$。
    - 具体来说，$\mathbf{h}_3 = \mathbf{W}_{hx} \mathbf{x}_3 + \mathbf{W}_{hh} \mathbf{h}_2$，这是公式8.7.9的第一部分。然后，$\mathbf{o}_3 = \mathbf{W}_{qh} \mathbf{h}_3$，这是公式8.7.9的第二部分。
  - 公式8.7.10是损失函数，表示所有时间步的损失的平均值，即$L = \frac{1}{T} \sum_{t=1}^T l(\mathbf{o}_t, y_t)$。
    - 此处的$l(\mathbf{o}_t, y_t)$是在时间步$t$的损失，$\mathbf{o}_t$是模型的输出，$y_t$是真实的标签。
  - 在训练过程中，需要计算损失函数关于模型参数的梯度，即$\partial L/\partial \mathbf{W}_{hx}$、$\partial L/\partial \mathbf{W}_{hh}$和$\partial L/\partial \mathbf{W}_{qh}$。
    - 该过程可通过反向传播算法实现，具体来说，就是沿着计算图的箭头反向遍历，依次计算和存储梯度。
  - 最后，$\text{prod}$运算符是用来表示链式法则中不同形状的矩阵、向量和标量的乘法。
    - 反向传播需要计算各种形状的张量的乘法，$\text{prod}$运算符就是用来完成这个任务的。


-------

首先，在任意时间步$t$，
目标函数关于模型输出的微分计算是相当简单的：

$$\frac{\partial L}{\partial \mathbf{o}_t} =  \frac{\partial l (\mathbf{o}_t, y_t)}{T \cdot \partial \mathbf{o}_t} \in \mathbb{R}^q \tag{8.7.11}$$

现在，我们可以计算目标函数关于输出层中参数$\mathbf{W}_{qh}$的梯度：
$\partial L/\partial \mathbf{W}_{qh} \in \mathbb{R}^{q \times h}$。
基于图8.7.2，
目标函数$L$通过$\mathbf{o}_1, \ldots, \mathbf{o}_T$
依赖于$\mathbf{W}_{qh}$。
依据链式法则，得到

$$
\frac{\partial L}{\partial \mathbf{W}_{qh}}
= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{W}_{qh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{o}_t} \mathbf{h}_t^\top \tag{8.7.12}
$$

其中$\partial L/\partial \mathbf{o}_t$是
由公式8.7.11给出的。

接下来，如 图8.7.2所示，
在最后的时间步$T$，目标函数$L$仅通过$\mathbf{o}_T$
依赖于隐状态$\mathbf{h}_T$。
因此，我们通过使用链式法可以很容易地得到梯度
$\partial L/\partial \mathbf{h}_T \in \mathbb{R}^h$：

$$\frac{\partial L}{\partial \mathbf{h}_T} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_T}, \frac{\partial \mathbf{o}_T}{\partial \mathbf{h}_T} \right) = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_T} \tag{8.7.13}$$


当目标函数$L$通过$\mathbf{h}_{t+1}$和$\mathbf{o}_t$
依赖$\mathbf{h}_t$时，
对于任意时间步$t < T$来说都变得更加棘手。
根据链式法则，隐状态的梯度
$\partial L/\partial \mathbf{h}_t \in \mathbb{R}^h$
在任何时间步骤$t < T$时都可以递归地计算为：

$$\frac{\partial L}{\partial \mathbf{h}_t} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_{t+1}}, \frac{\partial \mathbf{h}_{t+1}}{\partial \mathbf{h}_t} \right) + \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{h}_t} \right) = \mathbf{W}_{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+1}} + \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t} \tag{8.7.14}$$


为了进行分析，对于任何时间步$1 \leq t \leq T$展开递归计算得

$$\frac{\partial L}{\partial \mathbf{h}_t}= \sum_{i=t}^T {\left(\mathbf{W}_{hh}^\top\right)}^{T-i} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+t-i}} \tag{8.7.15}$$


我们可以从公式8.7.15中看到，
这个简单的线性例子已经展现了长序列模型的一些关键问题：
它陷入到$\mathbf{W}_{hh}^\top$的潜在的非常大的幂。
在这个幂中，小于1的特征值将会消失，大于1的特征值将会发散。
这在数值上是不稳定的，表现形式为梯度消失或梯度爆炸。
解决此问题的一种方法是按照计算方便的需要截断时间步长的尺寸
如8.7.1节中所述。
实际上，这种截断是通过在给定数量的时间步之后分离梯度来实现的。
稍后，我们将学习更复杂的序列模型（如长短期记忆模型）
是如何进一步缓解这一问题的。

-----------

- **说明：**
  - 公式8.7.13是在最后的时间步$T$，目标函数$L$通过输出$\mathbf{o}_T$依赖于隐状态$\mathbf{h}_T$。所以，我们可以使用链式法则来计算梯度$\partial L/\partial \mathbf{h}_T$。这个公式的功能是计算最后一个时间步的隐藏状态的梯度。
  - 公式8.7.14是对于任意时间步$t < T$，目标函数$L$通过$\mathbf{h}_{t+1}$和$\mathbf{o}_t$依赖$\mathbf{h}_t$。这个公式的功能是递归地计算任何时间步的隐藏状态的梯度。
   - 公式8.7.15是将公式8.7.14在任何时间步$1 \leq t \leq T$展开递归计算得到的结果。这个公式的功能是展示如何计算任何时间步的隐藏状态的梯度。



-------------

最后， 图8.7.2表明：
目标函数$L$通过隐状态$\mathbf{h}_1, \ldots, \mathbf{h}_T$
依赖于隐藏层中的模型参数$\mathbf{W}_{hx}$和$\mathbf{W}_{hh}$。
为了计算有关这些参数的梯度
$\partial L / \partial \mathbf{W}_{hx} \in \mathbb{R}^{h \times d}$和$\partial L / \partial \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$，
我们应用链式规则得：

$$
\begin{aligned}
\frac{\partial L}{\partial \mathbf{W}_{hx}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hx}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{x}_t^\top,\\
\frac{\partial L}{\partial \mathbf{W}_{hh}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{h}_{t-1}^\top,
\end{aligned} \tag{8.7.16}
$$

其中$\partial L/\partial \mathbf{h}_t$
是由公式8.7.13和公式8.7.14递归计算得到的，
是影响数值稳定性的关键量。

正如我们在4.7节中所解释的那样，
由于通过时间反向传播是反向传播在循环神经网络中的应用方式，
所以训练循环神经网络交替使用前向传播和通过时间反向传播。
通过时间反向传播依次计算并存储上述梯度。
具体而言，存储的中间值会被重复使用，以避免重复计算，
例如存储$\partial L/\partial \mathbf{h}_t$，
以便在计算$\partial L / \partial \mathbf{W}_{hx}$和
$\partial L / \partial \mathbf{W}_{hh}$时使用。

----------

- **说明：**
  - 公式8.7.16描述了如何计算循环神经网络（RNN）中模型参数$\mathbf{W}_{hx}$和$\mathbf{W}_{hh}$的梯度。这些梯度是训练RNN时用于参数更新的关键量。
  - 首先，目标函数$L$通过隐状态$\mathbf{h}_1, \ldots, \mathbf{h}_T$依赖于隐藏层中的模型参数$\mathbf{W}_{hx}$和$\mathbf{W}_{hh}$。这意味着目标函数的值会受到这些参数的影响。
  - 为了计算有关这些参数的梯度$\partial L / \partial \mathbf{W}_{hx}$和$\partial L / \partial \mathbf{W}_{hh}$，我们应用链式规则。链式规则是微积分中的一个基本原则，用于计算复合函数的导数。
    - 对于$\mathbf{W}_{hx}$的梯度$\partial L / \partial \mathbf{W}_{hx}$，我们将$L$关于$\mathbf{h}_t$的梯度$\partial L / \partial \mathbf{h}_t$与$\mathbf{h}_t$关于$\mathbf{W}_{hx}$的偏导数相乘，然后对所有的时间步$t$求和。这里的$\mathbf{x}_t$是在时间步$t$的输入。
    - 对于$\mathbf{W}_{hh}$的梯度$\partial L / \partial \mathbf{W}_{hh}$，我们将$L$关于$\mathbf{h}_t$的梯度$\partial L / \partial \mathbf{h}_t$与$\mathbf{h}_t$关于$\mathbf{W}_{hh}$的偏导数相乘，然后对所有的时间步$t$求和。这里的$\mathbf{h}_{t-1}$是在时间步$t-1$的隐藏状态。
    - 其中，$\partial L/\partial \mathbf{h}_t$是由公式8.7.13和公式8.7.14递归计算得到的，是影响数值稳定性的关键量。这意味着我们需要在每个时间步计算并存储这个量，然后在后续的计算中使用。
  - 在训练RNN时，我们通常使用前向传播和通过时间反向传播的交替方式。这意味着我们首先使用前向传播计算每个时间步的隐藏状态，然后使用通过时间反向传播计算梯度。
  - 在通过时间反向传播过程中，我们依次计算并存储上述梯度。为了避免重复计算，我们通常会存储中间值，例如$\partial L/\partial \mathbf{h}_t$，然后在计算$\partial L / \partial \mathbf{W}_{hx}$和$\partial L / \partial \mathbf{W}_{hh}$时使用。
  - $\frac{\partial L}{\partial \mathbf{W}_{hx}}$是目标函数$L$相对于参数$\mathbf{W}_{hx}$的梯度，它是所有时间步长$t$上$\frac{\partial L}{\partial \mathbf{h}_t}$与$\mathbf{x}_t^\top$的乘积的总和。
   - $\frac{\partial L}{\partial \mathbf{W}_{hh}}$是目标函数$L$相对于参数$\mathbf{W}_{hh}$的梯度，它是所有时间步长$t$上$\frac{\partial L}{\partial \mathbf{h}_t}$与$\mathbf{h}_{t-1}^\top$的乘积的总和。


-----------

## 小结

* “通过时间反向传播”仅仅适用于反向传播在具有隐状态的序列模型。
* 截断是计算方便性和数值稳定性的需要。截断包括：规则截断和随机截断。
* 矩阵的高次幂可能导致神经网络特征值的发散或消失，将以梯度爆炸或梯度消失的形式表现。
* 为了计算的效率，“通过时间反向传播”在计算期间会缓存中间值。


----
- **说明：公式8.7.4深入探讨**
- 在理解公式 (8.7.4) 的推导时，可以借助链式法则来理解如何递归地计算梯度。
- 可通过一个简单的多变量链式法则来解释递归关系的推导。
- 假设：
  - $ x = g(t) $ 是关于 $ t $ 的可微函数。
  - $ z = f(x, t) $ 是关于 $ x $ 和 $ t $ 的可微函数。
- 可以将 $ z $ 写成 $ z = f(g(t), t) $，其中 $ z $ 是 $ t $ 的复合函数。
- 根据链式法则，$ z $ 关于 $ t $ 的导数为：

  $$
  \frac{d z}{d t} = \frac{\partial z}{\partial x} \frac{d x}{d t} + \frac{\partial z}{\partial t}
  $$
  - 此处的“+”号表示有两个途径影响 $ z $，一个是通过 $ x $ 间接影响 $ z $，另一个是 $ t $ 直接影响 $ z $。
  - 第一项 $\frac{\partial z}{\partial x} \frac{d x}{d t}$ 表示 $ z $ 通过 $ x $ 间接对 $ t $ 的依赖。
  - 第二项 $\frac{\partial z}{\partial t}$ 表示 $ z $ 直接对 $ t $ 的依赖。

- 应用到RNN中的梯度计算有相似的情况：
  - $ h_t $ 是时间步 $ t $ 的隐状态，类似于上面的 $ z $。
  - $ h_{t-1} $ 是前一时间步的隐状态，类似于 $ x $。
  - $ w_h $ 是需要关注的参数，类似于上面的 $ t $。
- 根据递归神经网络的定义，$ h_t $ 是通过 $ h_{t-1} $ 和输入 $ x_t $ 计算得到的。
  $$
  h_t = f(x_t, h_{t-1}, w_h)
  $$
- 现在想要计算 $ h_t $ 对 $ w_h $ 的梯度。由于 $ h_t $ 直接依赖于 $ h_{t-1} $ 和 $ w_h $，并且 $ h_{t-1} $ 又递归地依赖于 $ w_h $，需要使用链式法则来计算梯度。

- 根据链式法则，$ h_t $ 对 $ w_h $ 的梯度为：
  $$
  \frac{\partial h_t}{\partial w_h} = \frac{\partial f(x_t, h_{t-1}, w_h)}{\partial w_h} + \frac{\partial f(x_t, h_{t-1}, w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}
  $$

  - 第一项 $\frac{\partial f(x_t, h_{t-1}, w_h)}{\partial w_h}$ 表示 $ h_t $ 直接对 $ w_h $ 的依赖。
  - 第二项 $\frac{\partial f(x_t, h_{t-1}, w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}$ 表示 $ h_t $ 间接通过 $ h_{t-1} $ 对 $ w_h $ 的依赖，而 $ h_{t-1} $ 自身也依赖于 $ w_h $。
- 公式的结构类别：
  - $ h_t $ 类似于 $ z $，
  - $ h_{t-1} $ 类似于 $ x $，
  - $ w_h $ 类似于 $ t $。
  - 由此得到了递归关系式 (8.7.4)，表明为了计算 $ h_t $ 对 $ w_h $ 的梯度，需要结合 $ h_t $ 对 $ w_h $ 的直接影响和 $ h_{t-1} $ 对 $ w_h $ 的间接影响。

---

- **说明：公式8.7.15的推导**

- 公式8.7.14 给出了递归形式的梯度计算公式。现在我们可以通过递归展开的方式，将其推广到更早的时间步。
- 从公式8.7.14出发，递归地计算 $ \frac{\partial L}{\partial \mathbf{h}_t} $：
  $$
  \frac{\partial L}{\partial \mathbf{h}_t} = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t} +   \mathbf{W}_{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+1}}
  $$

- 对于 $ \frac{\partial L}{\partial \mathbf{h}_{t+1}} $，根据同样的递归公式，我们可以继续展开：
  $$
  \frac{\partial L}{\partial \mathbf{h}_{t+1}} = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{t+1}} + \mathbf{W}_{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+2}}
  $$
- 将其代入到 $ \frac{\partial L}{\partial \mathbf{h}_t} $ 中：
  $$
  \frac{\partial L}{\partial \mathbf{h}_t} = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t} + \mathbf{W}_{hh}^\top \left( \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{t+1}} + \mathbf{W}_{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+2}} \right)
  $$
- 继续展开下去，直到时间步 $ T $，我们可以得到一个逐级累积的梯度表达式：
  $$
  \frac{\partial L}{\partial \mathbf{h}_t} = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t} + \mathbf{W}_{hh}^\top \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{t+1}} + \dots + \left( \mathbf{W}_{hh}^\top \right)^{T-t} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_T}
  $$

- 可以看到，这个公式中包含了从时间步 $ t $ 开始到时间步 $ T $ 的所有输出的梯度贡献，将其写成求和形式：
  $$
  \frac{\partial L}{\partial \mathbf{h}_t} = \sum_{i=t}^{T} \left( \mathbf{W}_{hh}^\top \right)^{i-t} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_i}
  $$
- 上述公式和公式8.7.15是一致的，只不过原公式是从t到T求和，而此公式是从T到t求和，求和公式中的项顺序是满足交换律的。
  $$\frac{\partial L}{\partial \mathbf{h}_t}= \sum_{i=t}^T {\left(\mathbf{W}_{hh}^\top\right)}^{T-i} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+t-i}} \tag{8.7.15}$$
- 交换律比如：
  - $\sum_{i=1}^n a_i = a_1 + a_2 + ... + a_n = a_n + a_{n-1} + ... + a_1$


----

- **附录：本节若干公式计算的示例代码**

- **（1）公式8.7.7：**$$\frac{\partial h_t}{\partial w_h}=\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_h)}{\partial h_{j-1}} \right) \frac{\partial f(x_{i},h_{i-1},w_h)}{\partial w_h} \tag{8.7.7}$$

In [ ]:
import numpy as np

def compute_gradient_8_7_7(T=3):
    """
    实现公式8.7.7的梯度计算
    T: 时间步数
    """
    np.random.seed(42)
    
    # 模型参数
    hidden_size = 4
    input_size = 3
    
    # 初始化参数
    wh = np.random.randn(hidden_size, hidden_size + input_size)
    
    # 模拟输入序列
    X = np.random.randn(T, input_size)
    
    def compute_f_and_derivatives(x_t, h_prev, wh):
        """
        计算f及其相关导数
        返回: f(x_t, h_t-1, wh), ∂f/∂w_h, ∂f/∂h_t-1
        """
        # 拼接输入和前一隐状态
        combined = np.concatenate([x_t, h_prev])
        
        # 计算f
        z = wh @ combined
        h_t = np.tanh(z)
        
        # 计算∂f/∂w_h
        df_dwh = np.outer((1 - h_t**2), combined)
        
        # 计算∂f/∂h_t-1
        df_dh = wh[:, input_size:] * (1 - h_t**2).reshape(-1, 1)
        
        return h_t, df_dwh, df_dh

    # 前向传播，存储中间结果
    h = np.zeros((T, hidden_size))
    df_dwh_list = []  # 存储所有时间步的∂f/∂w_h
    df_dh_list = []   # 存储所有时间步的∂f/∂h_t-1
    
    h_prev = np.zeros(hidden_size)
    for t in range(T):
        h_t, df_dwh, df_dh = compute_f_and_derivatives(X[t], h_prev, wh)
        h[t] = h_t
        df_dwh_list.append(df_dwh)
        df_dh_list.append(df_dh)
        h_prev = h_t

    # 实现公式8.7.7
    def compute_dh_dwh(t):
        """计算∂h_t/∂w_h"""
        if t == 0:
            return df_dwh_list[0]
        
        # 初始项：∂f(x_t,h_t-1,w_h)/∂w_h
        gradient = df_dwh_list[t]
        
        # 计算求和项
        for i in range(t):
            # 计算连乘项 ∏(∂f/∂h)
            product = np.eye(hidden_size)
            for j in range(i+1, t+1):
                product = product @ df_dh_list[j]
            
            # 乘以∂f/∂w_h并累加
            gradient += product @ df_dwh_list[i]
            
        return gradient

    # 计算每个时间步的梯度
    gradients = []
    for t in range(T):
        grad_t = compute_dh_dwh(t)
        gradients.append(grad_t)
        print(f"\n时间步 {t} 的梯度:")
        print(f"形状: {grad_t.shape}")
        print(f"部分值:\n{grad_t[:2, :2]}")
        print(f"统计信息:")
        print(f"最小值: {grad_t.min():.6f}")
        print(f"最大值: {grad_t.max():.6f}")
        print(f"平均值: {grad_t.mean():.6f}")
        print(f"标准差: {grad_t.std():.6f}")

    return gradients

# 运行计算
print("计算公式8.7.7的梯度...")
gradients = compute_gradient_8_7_7()

# 验证梯度
print("\n梯度健康检查:")
for t, grad in enumerate(gradients):
    print(f"\n时间步 {t}:")
    print(f"是否包含NaN: {np.isnan(grad).any()}")
    print(f"是否包含Inf: {np.isinf(grad).any()}")


- **（2）公式8.7.8：**
  $$z_t= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\xi_t \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h} \tag{8.7.8}$$

In [3]:
import numpy as np

def f(x, h, w):
    return np.dot(x, w) + h

def df(x, h, w):
    return np.dot(x, w), x

def random_truncation_gradient(x, h, w, pi):
    T = len(x)
    z = np.zeros_like(w)
    for t in range(T-1, -1, -1):
        df_dw, df_dh = df(x[t], h[t-1], w)
        xi_t = np.random.choice([0, 1/pi[t]], p=[1-pi[t], pi[t]])
        z = df_dw + xi_t * df_dh * z
    return z

# 生成仿真数据
T = 10
x = np.random.normal(size=(T, 5))
h = np.random.normal(size=(T, 5))
w = np.random.normal(size=(5, 5))
pi = np.random.uniform(size=T)

# 计算梯度
z = random_truncation_gradient(x, h, w, pi)
print(z)


[[ 3.94115535  0.31566167  1.50573174 -1.56613743  1.35916767]
 [ 3.94115535  0.31566167  1.50573174 -1.56613743  1.35916767]
 [ 3.94115535  0.31566167  1.50573174 -1.56613743  1.35916767]
 [ 3.94115535  0.31566167  1.50573174 -1.56613743  1.35916767]
 [ 3.94115535  0.31566167  1.50573174 -1.56613743  1.35916767]]


- **（3）公式8.7.15：**
  $$\frac{\partial L}{\partial \mathbf{h}_t}= \sum_{i=t}^T {\left(\mathbf{W}_{hh}^\top\right)}^{T-i} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+t-i}} \tag{8.7.15}$$

In [4]:
# 公式8.7.15
import numpy as np

# 设定随机种子以保证结果可复现
np.random.seed(0)

# 设定相关参数
T = 10  # 时间步数
h = 5  # 隐藏状态的维度

# 随机生成权重矩阵和梯度
W_qh = np.random.rand(h, h)
W_hh = np.random.rand(h, h)
grad_o = np.random.rand(T, h)
grad_h_next = np.zeros(h)

# 初始化隐藏状态的梯度
grad_h = np.zeros((T, h))

# 从最后一个时间步开始，反向遍历每个时间步
for t in range(T - 1, -1, -1):
    # 计算公式8.7.13和公式8.7.14
    if t == T - 1:
        grad_h[t] = np.dot(W_qh.T, grad_o[t])
    else:
        grad_h[t] = np.dot(W_hh.T, grad_h_next) + np.dot(W_qh.T, grad_o[t])
    grad_h_next = grad_h[t]

# 输出隐藏状态的梯度
print("隐藏状态的梯度：")
print(grad_h)

# 计算公式8.7.15
grad_h_expanded = np.zeros((T, h))
for t in range(T):
    for i in range(t, T):
        grad_h_expanded[t] += np.linalg.matrix_power(W_hh.T, i - t) @ W_qh.T @ grad_o[i]

# 输出展开后的隐藏状态的梯度
print("展开后的隐藏状态的梯度：")
print(grad_h_expanded)


隐藏状态的梯度：
[[6.81422558e+03 5.71935773e+03 7.69202633e+03 6.56437397e+03
  5.91126266e+03]
 [2.73878951e+03 2.29870964e+03 3.09213435e+03 2.63858823e+03
  2.37607036e+03]
 [1.10082327e+03 9.23993658e+02 1.24219638e+03 1.06066938e+03
  9.54863347e+02]
 [4.41576253e+02 3.70856772e+02 4.99758107e+02 4.26182193e+02
  3.83769410e+02]
 [1.77343914e+02 1.48835055e+02 1.99839268e+02 1.71369293e+02
  1.53140768e+02]
 [7.02079107e+01 5.89085734e+01 7.94102275e+01 6.78988892e+01
  6.11163875e+01]
 [2.78599999e+01 2.33600386e+01 3.18388397e+01 2.72574622e+01
  2.42966997e+01]
 [1.08721961e+01 8.96453528e+00 1.20060003e+01 1.06799970e+01
  9.19791155e+00]
 [3.52555959e+00 2.85820895e+00 4.21024679e+00 3.92444063e+00
  3.47594526e+00]
 [5.72102836e-01 4.18814122e-01 1.33723821e+00 1.33234028e+00
  1.02567492e+00]]
展开后的隐藏状态的梯度：
[[6.81422558e+03 5.71935773e+03 7.69202633e+03 6.56437397e+03
  5.91126266e+03]
 [2.73878951e+03 2.29870964e+03 3.09213435e+03 2.63858823e+03
  2.37607036e+03]
 [1.10082327e+03 

- **（4）公式8.7.16：**
  $$
\begin{aligned}
\frac{\partial L}{\partial \mathbf{W}_{hx}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hx}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{x}_t^\top,\\
\frac{\partial L}{\partial \mathbf{W}_{hh}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{h}_{t-1}^\top,
\end{aligned} \tag{8.7.16}
$$

In [5]:
# 公式8.7.16的计算
import numpy as np

# 仿真数据
T = 2  # 时间步长
d = 3  # 输入维度
h_dim = 4  # 隐藏状态维度

# 随机初始化参数
np.random.seed(0)
W_hx = np.random.randn(h_dim, d)
W_hh = np.random.randn(h_dim, h_dim)

# 随机生成输入数据
x = np.random.randn(T, d)
h = np.random.randn(T+1, h_dim)  # h[0] 是初始隐藏状态

# 随机生成目标函数L相对于每个时间步长的隐藏状态的梯度
dL_dh = np.random.randn(T, h_dim)

# 初始化梯度
dL_dW_hx = np.zeros_like(W_hx)
dL_dW_hh = np.zeros_like(W_hh)

# 计算梯度
for t in range(T):
    dL_dW_hx += np.outer(dL_dh[t], x[t])
    dL_dW_hh += np.outer(dL_dh[t], h[t-1])

print("dL/dW_hx:\n", dL_dW_hx)
print("dL/dW_hh:\n", dL_dW_hh)


dL/dW_hx:
 [[-2.25889058 -1.0458234   1.57961959]
 [ 1.33803308  0.79892575 -0.64590497]
 [-2.66691643 -1.91790944  0.7617317 ]
 [-0.77255472  0.73555663  2.3056285 ]]
dL/dW_hh:
 [[ 2.44915107 -2.58392764 -0.46319428 -0.5278735 ]
 [-1.4612167   1.57720086  0.07975288  0.12460522]
 [ 2.9314611  -3.22821607  0.19408776  0.09282537]
 [ 0.77374868 -0.59959913 -1.34409723 -1.3263723 ]]


---